In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")  # Run in background
driver = webdriver.Chrome(options=options)

query = "transformer models in NLP"
driver.get(f"https://scholar.google.com/scholar?q={query}")

pdf_links=[]
# Find PDF links (right-hand panel)
pdfs = driver.find_elements(By.XPATH, "//div[@class='gs_or_ggsm']/a")
for pdf in pdfs:
    url = pdf.get_attribute("href")
    print("PDF:", url)
    pdf_links.append(url)

driver.quit()


PDF: https://arodes.hes-so.ch/record/6812/files/published%20version.pdf
PDF: https://arxiv.org/pdf/2108.05542
PDF: https://www.mdpi.com/2673-2688/4/1/4/pdf
PDF: https://aclanthology.org/2020.emnlp-demos.6.pdf
PDF: https://www.researchgate.net/profile/Saurav-Singla-4/publication/346565049_Comparative_Analysis_of_Transformer_Based_Pre-Trained_NLP_Models/links/5fc77e3ea6fdcc697bd360ec/Comparative-Analysis-of-Transformer-Based-Pre-Trained-NLP-Models.pdf
PDF: https://www.mdpi.com/2078-2489/14/4/242/pdf
PDF: https://www.researchgate.net/profile/Henry-Nunoo-Mensah/publication/348740926_Transformer_Models_for_Text-based_Emotion_Detection_A_Review_of_BERT-based_Approaches/links/600e5ef145851553a06b0ac4/Transformer-Models-for-Text-based-Emotion-Detection-A-Review-of-BERT-based-Approaches.pdf
PDF: https://arxiv.org/pdf/1910.03771


In [20]:
import requests
import os
import re

SAVE_DIR = 'samples_pdf'
for url in pdf_links:
    filename = url.split("/")[-1]
    if not re.findall(".pdf", filename):
        filename = filename + ".pdf"
    filename = os.path.join(SAVE_DIR, filename)
    r = requests.get(url)
    with open(filename, "wb") as f:
        f.write(r.content)


In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--headless")  # No browser window
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
#url = "https://onlinelibrary.wiley.com/doi/abs/10.1002/ijc.29408"

query = "convolution neural network for WBC classification"
driver.get(f"https://scholar.google.com/scholar?q={query}")
time.sleep(3)

html = driver.page_source

with open("samples_pdf/real_page.html", "w", encoding="utf-8") as f:
    f.write(html)

driver.quit()

In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import time
import re

# Setup headless Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

query = "convolution neural network for WBC classification"
search_url = f"https://scholar.google.com/scholar?q={query}"
driver.get(search_url)
time.sleep(3)

# Create output folder
os.makedirs("samples_pdf", exist_ok=True)

results = driver.find_elements(By.CSS_SELECTOR, ".gs_r.gs_or.gs_scl")

def extract_citations_and_links(results):
    extracted = []
    for res in results:
        try:
            title = res.find_element(By.CSS_SELECTOR, "h3").text
            link_el = res.find_element(By.CSS_SELECTOR, "h3 a")
            article_link = link_el.get_attribute("href")

            cite_text = res.find_element(By.PARTIAL_LINK_TEXT, "Cited by").text
            citations = int(re.search(r"Cited by (\d+)", cite_text).group(1))

            extracted.append({
                "title": title,
                "citations": citations,
                "link": article_link
            })
        except Exception:
            continue
    return sorted(extracted, key=lambda x: -x['citations'])

top_articles = extract_citations_and_links(results)

# Utility: Check if content looks valid
def is_valid_content(html):
    invalid_markers = [
        "enable javascript", "verify you are human",
        "access denied", "robot check", "just a moment",
        "cloudflare", "404 not found", "site can’t be reached"
    ]
    lower_html = html.lower()
    return not any(marker in lower_html for marker in invalid_markers) and len(lower_html.strip()) > 5000

# Visit and save up to 5 valid articles
valid_count = 0
i = 0
while valid_count < 5 and i < len(top_articles):
    article = top_articles[i]
    print(f"🔍 Trying article #{i+1}: {article['title']} ({article['citations']} citations)")
    try:
        driver.get(article["link"])
        time.sleep(5)
        html = driver.page_source

        if is_valid_content(html):
            file_path = f"samples_pdf/article_{valid_count+1}.html"
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(html)
            print(f"✅ Valid content saved: {file_path}")
            valid_count += 1
        else:
            print("⚠️ Skipped: Content invalid or blocked")
    except Exception as e:
        print(f"❌ Error loading article: {e}")
    i += 1

driver.quit()

🔍 Trying article #1: White blood cells detection and classification based on regional convolutional neural networks (268 citations)
⚠️ Skipped: Content invalid or blocked
🔍 Trying article #2: Automatic detection and classification of leukocytes using convolutional neural networks (250 citations)
✅ Valid content saved: samples_pdf/article_1.html
🔍 Trying article #3: White blood cell classification and counting using convolutional neural network (152 citations)
⚠️ Skipped: Content invalid or blocked
🔍 Trying article #4: An efficient multi-level convolutional neural network approach for white blood cells classification (89 citations)
⚠️ Skipped: Content invalid or blocked
🔍 Trying article #5: Classification of white blood cell using convolution neural network (70 citations)
⚠️ Skipped: Content invalid or blocked
🔍 Trying article #6: White blood cells classification with deep convolutional neural networks (67 citations)
⚠️ Skipped: Content invalid or blocked
🔍 Trying article #7: WBCs-Net: 